In [1]:
import numpy as np

# Adaptive Gradient Descent Algorithm

Why do we need adaptive gradient descent?<br>
1. We want an adaptive learning rate at times. If the slope of the loss curve is very steep at some time,<br>
    we wish for a small learning rate whereas if erros surface at somt point flat, we want the learning rate to be higher
2. If a sparse feature turns out to be very imporatnt we want a higher lr for it
    - During gradient descent the probability of the feature to get updated is low as the batch selected may have all <br>
        0 values corresponding to the feature.
    - Even if there is an update the update could be very small as most of the example of the batch has very few non-zero values<br>
Hence we sould like to compensate this small gradient value with large learning rate.

**AdaGrad** <https://youtu.be/WSvxne3oGr0?list=PLZ2ps__7DhBZVxMrSkTIcG6zZBDKUXCnM&t=292> <br>
Based on the given premise we want to device an algorithm which can compensate a sparse feature with larger learning rate<br>
The update rule for ada grad is given by
$$
    v_t = v_{t-1} + (\nabla w_t)^2\\
    w_{t+1} = v_t - \frac{\eta}{\sqrt{v_t+\epsilon}}\nabla w_t
$$
$v_t$ tracks the history of updates. The neurons which gets updated rarely or get updated by small quantity are compensated in the second eq.<br>

DISADVANTAGES<br>
1. The optimization process is a bit slow as this method do not use concept of momentum.
2. After certain iteration when ther term $v_t$ accumulates some history, the learning process slows down as the value of $v_t$ starts to increase.<br>This is particularly true for dense feature. 


**RMSProp** <https://www.youtube.com/watch?v=ubOy0NPI2cY&list=PLZ2ps__7DhBZVxMrSkTIcG6zZBDKUXCnM&index=39> <br>
1. To reduce the monotonic decay of learning rate of adagrad, rmsprop introduced a scaling term. The equation is given by
$$
    v_t = \beta v_{t-1} + (1-\beta)(\nabla w_t)^2\\
    w_{t+1} = v_t - \frac{\eta}{\sqrt{v_t+\epsilon}}\nabla w_t
$$
1. Upon expanding the $v_t$ term we can see that more weightage is given to recent gradient values.
2. Unlike the adagrad the accumulation of weights ($v_t$) can decrease after some iteration. This leads to stability of learning rate.
3. With the stability of learning rate it brings the problem of oscillation around the local minima. This can be fixed with proper<br>
    initialization of initial learning rate.
4. Proper initialization of learning rate is not trivial. This is a trial and error process which is often not feasible.

**AdaDelta**<br>
1. Adaptive optimization algorithm which doesn't require a initial learning rate to function.